In [1044]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from config import api_key
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy
import datetime
%matplotlib inline

In [1045]:
quandl.ApiConfig.api_key = api_key
#Variance Risk Premium: https://sites.google.com/site/haozhouspersonalhomepage
#EOM Nominal S&P, Nominal Dividends / Nominal Risk Free Rate: http://www.hec.unil.ch/agoyal/
#Consumer Sentiment: http://www.sca.isr.umich.edu/

In [1046]:
"""
BUILD DATASET

Monthly >>>
Index: S&P 500 Index Data
D12: Dividends are 12-month moving sums of dividends paid on the S&P 500 index (Goyal, Welch) 
     -- Confirmed with Y Charts
Rfree: Nominal Risk Free Rate
"""
from pandas.tseries.offsets import MonthEnd

goyal = pd.read_excel('PredictorData2017.xlsx')
df = pd.DataFrame(goyal)

#New DF with cleaned up Columns
df_cols = ['yyyymm', 'Index', 'D12', 'Rfree']
df = df[df_cols]

#Convert yyyymm to Datetime
yyyymm = pd.to_datetime(df['yyyymm'], format='%Y%m').dt.strftime('%Y-%m')
df['yyyymm'] = pd.to_datetime(yyyymm, format='%Y-%m') + MonthEnd(1)
df.set_index('yyyymm', inplace=True)
df.index.rename('Date', inplace=True);

#Drop rows prior to 1990-01
df = df['1989':'2019'];

In [1047]:
"""
2018 Data
"""
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf

start_sp = datetime.datetime(2018, 1, 1)
end_sp = datetime.datetime(2018, 12, 31)
interval = '1mo'

yf.pdr_override()
sp500 = pdr.get_data_yahoo('^GSPC', 
                           start_sp,
                             end_sp,
                              interval=interval)
    

#2018 S&P Index Data from Yahoo
#f2018 = '^GSPC.csv'
#data = pd.read_csv(f2018)
#sp18 = pd.DataFrame(data)
#sp18 = sp18[['Date', 'Close']]

sp500 = sp500[['Close']]
sp500 = sp500.rename(columns={'Close': 'Index'})
sp500.index = pd.to_datetime(sp500.index) + MonthEnd(1)
#sp500 = sp500.resample('M', kind='timestamp').agg({'Index': 'last'})

'\n2018 Data\n'

[*********************100%***********************]  1 of 1 downloaded


In [1048]:
#Merge 2018 DF with Main DF
df = pd.merge(df, sp500, how='outer', on=['Date', 'Index'])

In [1049]:
ticker = 'SPY'
transform = 'normalize'
start_date = '1989-01-01'
end_date = '2018-12-31'
period = 'monthly'

In [1050]:
#Access data from Quandl - 12 Months of Risk Free Rate Data
ticker_data = quandl.get('EOD/' + ticker, start_date=start_date, end_date=end_date, collapse=period)
risk_free_df = quandl.get('FRED/DTB3', start_date='2018-01-01', end_date='2018-12-31', collapse=period)
fed_funds_rate = quandl.get('FRED/FEDFUNDS', start_date=start_date, end_date=end_date, collapse=period)

In [1051]:
#Create DF with Risk Free Monthly Rate & 
risk_free_df.eval('Rfree = (Value / 12) / 100', inplace=True)
risk_free_df = risk_free_df[['Rfree']]
risk_free_df.index = pd.to_datetime(risk_free_df.index)

In [1052]:
#Merge Index DF with risk_free_rate DF
df = df.fillna(risk_free_df)

In [1053]:
#Create Fed Funds DF & Rename Column
fed_funds_df = pd.DataFrame(fed_funds_rate)
fed_funds_df = fed_funds_df.rename(columns={'Value': 'fed_funds'})

In [1054]:
#Merge Index DF with fed_funds DF
df = pd.merge(df, fed_funds_df, how='outer', on='Date')

In [1055]:
"""
VRP: Variance Risk Premium = IV - RV
IV: Implied Variance = VIX-Squared/12
RV: Realized Variance = Sum of Squared 5-minute log returns of the S&P 500 Index
"""
vrp = pd.read_fwf('VRPtable.txt')
df_vrp = pd.DataFrame(vrp);

In [1056]:
#Parse VRP Date & Create New Column
df_vrp['Date'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(df_vrp.Year, df_vrp.Month)]) + MonthEnd(1)

df_vrp.set_index('Date', inplace=True)
df_vrp = df_vrp[['VRP']]

In [1057]:
#Merge Index DF with VRP DF
df = pd.merge(df, df_vrp, how='outer', on='Date')

In [1058]:
#Import Consumer Sentiment Data
sentiment_file = 'tbmics.csv'
sentiment = pd.read_csv(sentiment_file)

sentiment['Date'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(sentiment.YYYY, sentiment.Month)]) + MonthEnd(1)
sentiment.set_index('Date', inplace=True)
sentiment = sentiment[['ICS_ALL']]
sentiment = sentiment['1989':'2018']

In [1059]:
#Merge Consumer Data
df = pd.merge(df, sentiment, how='outer', on='Date')

In [1060]:
#Import, parse and clean 2018 Dividend History
div18_file = 'Dividend_History.csv'
div18_data = pd.read_csv(div18_file)

div18 = pd.DataFrame(div18_data)
div18 = div18[['Date', 'Value']]
div18['Date'] = pd.to_datetime(div18['Date'])
div18.set_index('Date', inplace=True)
div18 = div18.rename(columns={'Value': 'D12'})

In [1061]:
#Add 2018 Div History to DF
df = df.fillna(div18)

In [1062]:
df.head()
df.tail()

,Index,D12,Rfree,fed_funds,VRP,ICS_ALL
Date,,,,,,
1989-01-31,297.47,9.837,0.006892,9.12,NaN,97.9
1989-02-28,288.86,9.924,0.007108,9.36,NaN,95.4
1989-03-31,294.87,10.011,0.007350,9.85,NaN,94.3
1989-04-30,309.64,10.130,0.007208,9.84,NaN,91.5
1989-05-31,320.52,10.249,0.007025,9.81,NaN,90.7


,Index,D12,Rfree,fed_funds,VRP,ICS_ALL
Date,,,,,,
2018-08-31,2901.52,52.31,0.001725,1.91,8.5030,96.2
2018-09-30,2913.98,52.70,0.001792,1.95,7.9528,100.1
2018-10-31,2711.74,53.08,0.001908,2.19,-2.2509,98.6
2018-11-30,2760.17,53.74,0.001933,2.20,2.7329,97.5
2018-12-31,2506.85,54.38,0.002000,2.27,-4.0034,98.3


In [1091]:
def build_df(df):
    
    #Number of Periods in Variance Risk Premium
    vrp_rolling_periods = 3

    #Create Column with PD Ratio
    df['pd'] = (df['Index'] / df['D12'])

    #Create Column with Fed Funds Rate Delta (12 Month)
    df['ff12_D'] = df['fed_funds'] - df['fed_funds'].shift(12)

    #Create Column with Rolling Variance Risk Premium Rolling
    df['vrp3'] = df['VRP'].rolling(vrp_rolling_periods).mean()

    #Create Column with Sentiment Delta
    df['sent12_D'] = (df['ICS_ALL'] - df['ICS_ALL'].shift(12))
    
    #Create Column with Excess Return on Index Relative to the RFR in percent
    df['ersf'] = (((df['Index'] + df['D12']/12 - df['Index'].shift(1)) / df['Index'].shift(1)) - df['Rfree']) * 100

    #Create Column with Excess Return on Index Relative to the RFR in percent
    df['ersf_t1'] = (((df['Index'].shift(-1) + df['D12'].shift(-1)/12 - df['Index']) / df['Index']) - df['Rfree'].shift(-1)) * 100

    #Create Column with 1-month Delta in ersf
    df['ersf_D'] = df['ersf'] - df['ersf'].shift(1)

    #Create column with Sentiment X ersf Delta
    df['sent_x_ersf_D'] = df['sent12_D'] * df['ersf_D']
    
    return df

df_all = build_df(df)
df_all = df_all['1990-03-01':'2018']

In [1092]:
df_all.head()
df_all.tail()

,Index,D12,Rfree,fed_funds,VRP,ICS_ALL,pd,ff12_D,vrp3,sent12_D,ersf,ersf_t1,ersf_D,sent_x_ersf_D
Date,,,,,,,,,,,,,,
1990-03-31,339.94,11.318,0.006583,8.28,26.5978,91.3,30.035342,-1.57,31.806967,-3.0,2.051350,-3.055940,1.558067,-4.674200
1990-04-30,330.80,11.433,0.006475,8.26,26.2753,93.9,28.933788,-1.58,28.596933,2.4,-3.055940,8.844823,-5.107289,-12.257494
1990-05-31,361.23,11.548,0.006450,8.18,19.2201,90.6,31.280741,-1.63,24.031067,-0.1,8.844823,-1.263740,11.900762,-1.190076
1990-06-30,358.02,11.663,0.006442,8.29,13.0796,88.3,30.697076,-1.24,19.525000,-2.3,-1.263740,-0.884521,-10.108562,23.249693
1990-07-31,356.15,11.720,0.006350,8.15,26.9870,88.2,30.388225,-1.09,19.762233,-3.8,-0.884521,-9.776690,0.379219,-1.441033


,Index,D12,Rfree,fed_funds,VRP,ICS_ALL,pd,ff12_D,vrp3,sent12_D,ersf,ersf_t1,ersf_D,sent_x_ersf_D
Date,,,,,,,,,,,,,,
2018-08-31,2901.52,52.31,0.001725,1.91,8.5030,96.2,55.467788,0.75,9.972800,-0.6,3.008606,0.401621,-0.586791,0.352075
2018-09-30,2913.98,52.70,0.001792,1.95,7.9528,100.1,55.293738,0.80,8.198900,5.0,0.401621,-6.979372,-2.606985,-13.034925
2018-10-31,2711.74,53.08,0.001908,2.19,-2.2509,98.6,51.087792,1.04,4.734967,-2.1,-6.979372,1.757751,-7.380993,15.500086
2018-11-30,2760.17,53.74,0.001933,2.20,2.7329,97.5,51.361556,1.04,2.811600,-1.0,1.757751,-9.213515,8.737123,-8.737123
2018-12-31,2506.85,54.38,0.002000,2.27,-4.0034,98.3,46.098750,0.97,-1.173800,2.4,-9.213515,NaN,-10.971266,-26.331038


In [1094]:
#Create DF for 2004 to 2018
df0418 = df_all['2004':'2018']
df0418.head()
df0418.tail()

,Index,D12,Rfree,fed_funds,VRP,ICS_ALL,pd,ff12_D,vrp3,sent12_D,ersf,ersf_t1,ersf_D,sent_x_ersf_D
Date,,,,,,,,,,,,,,
2004-01-31,1131.13,17.598000,0.000733,1.00,14.0860,103.8,64.276054,-0.24,16.151067,21.4,1.786198,1.274621,-3.352254,-71.738239
2004-02-29,1144.94,17.811000,0.000775,1.01,9.8728,94.4,64.282747,-0.25,14.917533,14.5,1.274621,-1.583041,-0.511577,-7.417861
2004-03-31,1126.21,18.024000,0.000783,1.00,9.4462,95.8,62.483910,-0.25,11.135000,18.2,-1.583041,-1.622623,-2.857662,-52.009454
2004-04-30,1107.30,18.216667,0.000783,1.00,13.4427,94.2,60.784995,-0.26,10.920567,8.2,-1.622623,1.261890,-0.039582,-0.324572
2004-05-31,1120.68,18.409333,0.000850,1.00,5.2403,90.2,60.875643,-0.26,9.376400,-1.9,1.261890,1.831398,2.884513,-5.480574


,Index,D12,Rfree,fed_funds,VRP,ICS_ALL,pd,ff12_D,vrp3,sent12_D,ersf,ersf_t1,ersf_D,sent_x_ersf_D
Date,,,,,,,,,,,,,,
2018-08-31,2901.52,52.31,0.001725,1.91,8.5030,96.2,55.467788,0.75,9.972800,-0.6,3.008606,0.401621,-0.586791,0.352075
2018-09-30,2913.98,52.70,0.001792,1.95,7.9528,100.1,55.293738,0.80,8.198900,5.0,0.401621,-6.979372,-2.606985,-13.034925
2018-10-31,2711.74,53.08,0.001908,2.19,-2.2509,98.6,51.087792,1.04,4.734967,-2.1,-6.979372,1.757751,-7.380993,15.500086
2018-11-30,2760.17,53.74,0.001933,2.20,2.7329,97.5,51.361556,1.04,2.811600,-1.0,1.757751,-9.213515,8.737123,-8.737123
2018-12-31,2506.85,54.38,0.002000,2.27,-4.0034,98.3,46.098750,0.97,-1.173800,2.4,-9.213515,NaN,-10.971266,-26.331038


In [1095]:
#Import Google Trends and Clean DF
google_trends_file = 'GoogleTrends_2004-2018.csv'

google_data = pd.read_csv(google_trends_file)
google = pd.DataFrame(google_data)
google = google[1:]

google = google.rename(columns={'Category: All categories': 'Google'})
google['Google'] = google['Google'].astype('float64')
google['Google_D'] = google['Google'] - google['Google'].shift(1)
google.fillna(google.mean(), inplace=True)

google.index = pd.to_datetime(google.index) + MonthEnd(1)
google.index.rename('Date', inplace=True);

In [1096]:
df0418 = pd.merge(df0418, google, how='outer', on='Date')

In [1097]:
#Create Delta for Google Delta Column
df0418['Google_D2'] = df0418['Google_D'] - df0418['Google_D'].shift(1)

In [1098]:
#Clean NaN Value
df0418['Google_D2'] = df0418['Google_D2'].fillna(df0418['Google_D2'].mean())
df0418.head()

,Index,D12,Rfree,fed_funds,VRP,ICS_ALL,pd,ff12_D,vrp3,sent12_D,ersf,ersf_t1,ersf_D,sent_x_ersf_D,Google,Google_D,Google_D2
Date,,,,,,,,,,,,,,,,,
2004-01-31,1131.13,17.598000,0.000733,1.00,14.0860,103.8,64.276054,-0.24,16.151067,21.4,1.786198,1.274621,-3.352254,-71.738239,28.0,0.273743,0.160482
2004-02-29,1144.94,17.811000,0.000775,1.01,9.8728,94.4,64.282747,-0.25,14.917533,14.5,1.274621,-1.583041,-0.511577,-7.417861,28.0,0.000000,-0.273743
2004-03-31,1126.21,18.024000,0.000783,1.00,9.4462,95.8,62.483910,-0.25,11.135000,18.2,-1.583041,-1.622623,-2.857662,-52.009454,26.0,-2.000000,-2.000000
2004-04-30,1107.30,18.216667,0.000783,1.00,13.4427,94.2,60.784995,-0.26,10.920567,8.2,-1.622623,1.261890,-0.039582,-0.324572,25.0,-1.000000,1.000000
2004-05-31,1120.68,18.409333,0.000850,1.00,5.2403,90.2,60.875643,-0.26,9.376400,-1.9,1.261890,1.831398,2.884513,-5.480574,23.0,-2.000000,-1.000000


In [1101]:
df_all.reset_index(inplace=True)
df0418.reset_index(inplace=True)

In [1102]:
export_csv = df_all.to_csv(r'/Users/maks_p/Flatiron/PROJECT/momentum_sentiment_FINAL/1989_2018_Data.csv', index=False);

In [1109]:
export_csv = df0418.to_csv(r'/Users/maks_p/Flatiron/PROJECT/momentum_sentiment_FINAL/2004_to_2018_withGoogle.csv', index=False);